# Neo4j와 LangChain을 활용한 뉴스 데이터 분석

---

## 1. Neo4J Desktop 환경 설정

- **Neo4j Desktop 소개**:
    - Neo4j 작업을 위한 클라이언트 애플리케이션
    - 로컬 환경에서 Neo4j를 학습하고 실험하는 데 필요한 모든 것을 포함함
    - 사용자의 컴퓨터 리소스가 허용하는 한 **여러 로컬 데이터베이스**를 생성할 수 있음
    - **Enterprise Edition 라이센스**: 단, 개발자 개인에 대해서는 1개 계정을 테스트 목적으로 지원

- **다운로드 및 설치**: https://neo4j.com/deployment-center/?desktop-gdb
    - Neo4J 5.24.0 선택
    - 새 프로젝트 생성 및 DBMS 추가

- **APOC 플러그인 설정**: 
    - APOC 플러그인을 설치하려는 데이터베이스가 있는 프로젝트(Graph DBMS)를 선택
    - Graph DBMS 메뉴 클릭하고, APOC 플러그인(Plugin) 설치

- **설정 파일 수정**: 데이터베이스를 중지한 상태에서 데이터베이스 카드의 오른쪽에 있는 `...` (메뉴) 버튼을 클릭

    - 메뉴에서 **Settings** 선택하고 다음 내용을 추가 (`neo4j.conf` 파일)
        ```
        dbms.security.procedures.unrestricted=apoc.meta.*,apoc.*
        ```

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True,
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
CREATE (n:Test {name: "Hello Neo4j Desktop News DB"}) 
RETURN n
"""

graph.query(cypher_query)

[{'n': {'name': 'Hello Neo4j Desktop News DB'}}]

In [4]:
def reset_database(graph):
    """
    APOC 없이 데이터베이스 초기화하기
    """
    # 모든 노드와 관계 삭제
    graph.query("MATCH (n) DETACH DELETE n")
    
    # 모든 제약조건 삭제
    constraints = graph.query("SHOW CONSTRAINTS")
    for constraint in constraints:
        constraint_name = constraint.get("name")
        if constraint_name:
            graph.query(f"DROP CONSTRAINT {constraint_name}")
    
    # 모든 인덱스 삭제
    indexes = graph.query("SHOW INDEXES")
    for index in indexes:
        index_name = index.get("name")
        index_type = index.get("type")
        if index_name and index_type != "CONSTRAINT":
            graph.query(f"DROP INDEX {index_name}")
    
    print("데이터베이스가 초기화되었습니다.")

# 데이터베이스 초기화
reset_database(graph)

데이터베이스가 초기화되었습니다.


---

## 2. **Knowledge Graph 구축**

### 2.1 뉴스 데이터 전처리


#### 1) **데이터셋 준비**

In [5]:
# 뉴스 데이터 로드
def load_news_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        news_article = file.read()
    return news_article

# 뉴스 데이터 목록 확인 (glob)
import glob
news_files = sorted(glob.glob(os.path.join('news_data', '*.md')))

# 뉴스 데이터 목록 출력
print("뉴스 데이터 목록:")
for news_file in news_files:
    print(os.path.basename(news_file))

# 뉴스 데이터 로드 및 정리
news_articles = []
for news_file in news_files:
    news_article = load_news_data(news_file)
    news_articles.append(news_article)

# 뉴스 데이터 출력
print("\n뉴스 데이터:")
for i, news_article in enumerate(news_articles):
    print(f"\n뉴스 기사 {i + 1}:\n{news_article[:100]}...")  # 첫 100자만 출력
    print("-" * 100)


뉴스 데이터 목록:
news_article_1.md
news_article_2.md
news_article_3.md
news_article_4.md
news_article_5.md

뉴스 데이터:

뉴스 기사 1:
# 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재

**디지털타임스 | 박승리 기자 | 2024-03-20**

구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고...
----------------------------------------------------------------------------------------------------

뉴스 기사 2:
# 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화

**디지털타임스 | 최기상 기자 | 2024-03-19**

애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI ...
----------------------------------------------------------------------------------------------------

뉴스 기사 3:
# 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개

**디지털타임스 | 정주리 기자 | 2024-04-05**

메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공...
----------------------------------------------------------------------------------------------------

뉴스 기사 4:
# 삼성전자, 'AI 메모리' 신기술 개발 성공

**전자일보 | 이승지 기자 | 2024-04-03**

삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적...
----------------------------------------------------------------------------------------------------

뉴스 기사 5:
# 엔비디아, 차세대 AI 전용 GP

#### 2) **뉴스 메타데이터 추출**

- 랭체인 구조화 출력 활용 (LLM 활용)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from datetime import date

# 뉴스 기사 메타데이터 모델 정의
class NewsArticleMetadata(BaseModel):
    """뉴스 기사의 구조화된 메타데이터."""
    title: str = Field(description="뉴스 기사의 제목")
    source: str = Field(description="뉴스 출처/발행처")
    author: str = Field(description="기사 작성자의 이름 (직위 등 표시 불필요)")
    date: str = Field(description="발행일(YYYY-MM-DD 형식)")
    content: str = Field(description="기사의 전체 텍스트 내용")

def extract_article_metadata(article_text: str) -> NewsArticleMetadata:
    """
    Langchain을 사용하여 뉴스 기사에서 구조화된 메타데이터를 추출합니다.
    
    Args:
        article_text (str): 뉴스 기사의 전체 텍스트
    
    Returns:
        NewsArticleMetadata: 기사에서 추출한 구조화된 메타데이터
    """

    # 프롬프트 템플릿 정의 - LLM에게 메타데이터 추출 지시사항 제공
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert news article metadata extractor. 
        Extract precise and accurate information from the given news article.
        
        Extraction Guidelines:
        - Identify the most accurate title
        - Determine the primary source/publication
        - Extract the author's name (only name, no position)
        - Identify the publication date
        - List key organizations mentioned
        - Highlight key technologies discussed
        
        Be as specific and factual as possible."""),
        ("human", """Extract metadata from the following article:\n\n{article_text}""")
    ])

    # LLM 및 체인 설정
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)    

    # 구조화된 출력을 위해 Pydantic 모델과 연결
    llm_with_structured_output = llm.with_structured_output(NewsArticleMetadata)
    
    # 프롬프트와 LLM을 연결하는 체인 구성
    chain = prompt | llm_with_structured_output
    
    # 메타데이터 추출 시도
    try:
        # LLM을 통해 메타데이터 추출
        metadata = chain.invoke({
            "article_text": article_text,
        })
        
        # 데이터 누락 시, 기본값 설정 - 데이터 일관성 유지
        if not metadata.title:
            metadata.title = ""
        if not metadata.source:
            metadata.source = "미상"
        if not metadata.author:
            metadata.author = "미상"
        if not metadata.date:
            metadata.date = "미상"
        if not metadata.content:
            metadata.content = article_text
        
        return metadata
    except Exception as e:
        # 에러 발생 시 로그 출력 및 기본 메타데이터 반환
        print(f"메타데이터 추출 중 오류 발생: {e}")
        return NewsArticleMetadata(
            title="",
            source="미상",
            author="미상",
            date="미상",
            content=""
        )

# 모든 뉴스 기사에서 메타데이터 추출 및 결과 출력
extracted_metadata = []
for article in news_articles:
    # 각 기사에서 메타데이터 추출
    metadata = extract_article_metadata(article)
    # 추출된 메타데이터 저장
    extracted_metadata.append(metadata)
    # 추출 결과 출력 - 디버깅 및 확인용
    print(f"Extracted metadata for article: {metadata.title}")
    print(f"Source: {metadata.source}")
    print(f"Author: {metadata.author}")
    print(f"Date: {metadata.date}")
    print(f"Content: {metadata.content[:100]}...")  # 첫 100자만 출력하여 가독성 확보

KeyboardInterrupt: 

In [ ]:
extracted_metadata[0].model_dump()

{'title': '구글, 안드로이드 14에 생성형 AI 기능 대거 탑재',
 'source': '디지털타임스',
 'author': '박승리',
 'date': '2024-03-20',
 'content': '구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활용한 사진 보정 기능이 크게 개선되었으며, 사용자가 촬영한 사진을 인공지능이 자동으로 분석하여 최적의 색감과 구도로 보정해주는 기능이 추가되었다. 또한 실시간 번역 기능도 대폭 강화되어 외국어 텍스트를 카메라로 비추기만 해도 즉시 번역 결과를 확인할 수 있고, 통화 중에도 실시간 음성 번역 기능이 추가되었다. 구글 어시스턴트는 생성형 AI 기술을 적용해 더 자연스러운 대화가 가능해졌으며, 복잡한 질문에 정확한 답변을 제공하고 개인화된 서비스를 제공한다. 구글 안드로이드 부문 책임자는 이번 AI 기능이 사용자 경험을 혁신적으로 개선할 것이라고 밝혔다. 업계 전문가들은 이번 AI 기능 강화가 모바일 OS 시장에서 경쟁력을 높이는 중요한 전환점이 될 것으로 전망하고 있다.'}

In [ ]:
# 메타데이터를 JSONL 형식으로 저장
import json
with open('news_data/news_metadata.jsonl', 'w', encoding='utf-8') as f:
    for metadata in extracted_metadata:
        json.dump(metadata.model_dump(), f, ensure_ascii=False)
        f.write('\n')

In [ ]:
# JSONL 파일 확인
import json
with open('news_data/news_metadata.jsonl', 'r', encoding='utf-8') as f:
    news_metadata = [json.loads(line) for line in f]
    print("뉴스 메타데이터:")
    for i, metadata in enumerate(news_metadata):
        print(f"\n뉴스 기사 {i + 1}:")
        print(f"Title: {metadata['title']}")
        print(f"Source: {metadata['source']}")
        print(f"Author: {metadata['author']}")
        print(f"Date: {metadata['date']}")
        print(f"Content: {metadata['content'][:100]}...")  # 첫 100자만 출력
        print("-" * 100)

뉴스 메타데이터:

뉴스 기사 1:
Title: 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재
Source: 디지털타임스
Author: 박승리
Date: 2024-03-20
Content: 구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활...
----------------------------------------------------------------------------------------------------

뉴스 기사 2:
Title: 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화
Source: 디지털타임스
Author: 최기상
Date: 2024-03-19
Content: 애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다. 최근 샌프란시스코에서 열린 WWDC 2024에서 공개된 iOS 18은 애플의 인공지능 전...
----------------------------------------------------------------------------------------------------

뉴스 기사 3:
Title: 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개
Source: 디지털타임스
Author: 정주리
Date: 2024-04-05
Content: 메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다 2배 큰 파라미터 규모와 10배 많은 학습 데이터를 기반으로 개발되었다고 회사 측은 밝혔...
----------------------------------------------------------------------------------------------------

뉴스 기사 4:
Title: 삼성전자, 'AI 메모리' 신기술 개발 성공
Source: 전자일보
Author: 이승지
Date: 

#### 3) **뉴스 데이터 구조화**

- 추출한 메타데이터 결합

In [ ]:
# 뉴스 데이터 구조화 - JSONL 파일에서 로드한 메타데이터를 분석 가능한 형태로 변환

extracted_articles = []  # 구조화된 뉴스 기사를 저장할 빈 리스트 초기화

# 각 뉴스 메타데이터를 순회하면서 구조화된 형태로 변환
for i, article in enumerate(news_metadata):
    
    # 개별 기사 데이터 구조화 - 일관된 형식으로 데이터 정리
    # - id: 고유 식별자 부여 (article_0, article_1 등의 형식)
    # - 제목, 출처, 작성자, 날짜, 내용 등 핵심 정보 포함
    article = {
        "id": f"article_{i}",  # 고유 식별자 생성 (인덱스 기반)
        "title": article["title"],  # 기사 제목
        "source": article["source"],  # 기사 출처 (언론사)
        "author": article["author"],  # 기사 작성자
        "date": article["date"],  # 기사 발행일
        "content": article["content"]  # 기사 전체 내용
    }

    # 구조화된 기사 데이터를 리스트에 추가 - 이후 지식 그래프 구축에 활용
    extracted_articles.append(article)

# 구조화된 기사 데이터 확인 - 처리 결과 검증 및 디버깅
print("구조화된 뉴스 기사 데이터:")
for article in extracted_articles:
    print(f"ID: {article['id']}")  # 고유 식별자 출력
    print(f"Title: {article['title']}")  # 기사 제목 출력
    print(f"Source: {article['source']}")  # 출처 출력
    print(f"Author: {article['author']}")  # 작성자 출력
    print(f"Date: {article['date']}")  # 날짜 출력
    print(f"Content: {article['content'][:100]}...")  # 내용 일부만 출력하여 가독성 확보
    print("-" * 100)

구조화된 뉴스 기사 데이터:
ID: article_0
Title: 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재
Source: 디지털타임스
Author: 박승리
Date: 2024-03-20
Content: 구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활...
----------------------------------------------------------------------------------------------------
ID: article_1
Title: 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화
Source: 디지털타임스
Author: 최기상
Date: 2024-03-19
Content: 애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다. 최근 샌프란시스코에서 열린 WWDC 2024에서 공개된 iOS 18은 애플의 인공지능 전...
----------------------------------------------------------------------------------------------------
ID: article_2
Title: 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개
Source: 디지털타임스
Author: 정주리
Date: 2024-04-05
Content: 메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다 2배 큰 파라미터 규모와 10배 많은 학습 데이터를 기반으로 개발되었다고 회사 측은 밝혔...
----------------------------------------------------------------------------------------------------
ID: article_3
Title: 삼성전자, 'AI 메모리' 신기술 개발 성공
Source: 전

### 2.2 KG 온톨로지 구현


#### 1) **스키마 정의**

- **노드 유형**:

    1. `NewsArticle`: 뉴스 기사
    2. `Company`: 회사
    3. `Product`: 제품
    4. `Technology`: 기술


- **관계 유형**:

    - `(NewsArticle)-[:MENTIONS]->(Entity)`
    - `(Company)-[:RELEASED]->(Product)`
    - `(Company)-[:DEVELOPED]->(Technology)`
    - `(Product)-[:USES]->(Technology)`

#### 2) **제약조건 설정**

In [ ]:
# Neo4j 데이터베이스에 Cypher 쿼리를 사용하여 제약조건 설정
# 제약조건은 노드의 특정 속성이 고유(UNIQUE)하도록 보장하여 데이터 중복을 방지함
constraints = [
    # NewsArticle 노드의 id 속성이 고유하도록 제약조건 설정
    # 이를 통해 동일한 id를 가진 뉴스 기사가 중복 저장되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (n:NewsArticle) REQUIRE n.id IS UNIQUE",
    
    # Company 노드의 name 속성이 고유하도록 제약조건 설정
    # 동일한 이름의 회사가 여러 번 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Company) REQUIRE c.name IS UNIQUE",
    
    # Product 노드의 name 속성이 고유하도록 제약조건 설정
    # 동일한 이름의 제품이 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Product) REQUIRE p.name IS UNIQUE",
    
    # Technology 노드의 name 속성이 고유하도록 제약조건 설정
    # 동일한 이름의 기술이 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Technology) REQUIRE t.name IS UNIQUE",
    
    # Person 노드의 name 속성이 고유하도록 제약조건 설정
    # 동일한 이름의 인물이 중복 생성되는 것을 방지
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.name IS UNIQUE",
]

# 정의된 모든 제약조건을 순회하며 Neo4j 데이터베이스에 적용
# graph.query() 메서드를 사용하여 각 Cypher 쿼리를 실행
for constraint in constraints:
    graph.query(constraint)
    

In [ ]:
graph.query("SHOW CONSTRAINTS")  # 현재 설정된 제약조건 확인

[{'id': 9,
  'name': 'constraint_43bf92ef',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Person'],
  'properties': ['name'],
  'ownedIndex': 'constraint_43bf92ef',
  'propertyType': None},
 {'id': 7,
  'name': 'constraint_676af2eb',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Technology'],
  'properties': ['name'],
  'ownedIndex': 'constraint_676af2eb',
  'propertyType': None},
 {'id': 0,
  'name': 'constraint_8b429a9c',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['NewsArticle'],
  'properties': ['id'],
  'ownedIndex': 'constraint_8b429a9c',
  'propertyType': None},
 {'id': 3,
  'name': 'constraint_bc3ece84',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Company'],
  'properties': ['name'],
  'ownedIndex': 'constraint_bc3ece84',
  'propertyType': None},
 {'id': 5,
  'name': 'constraint_ef0216e9',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Product'],
  'properties': 

#### 3) **뉴스 기사에서 엔티티와 관계를 추출**

- 랭체인 `LLMGraphTransformer` 초기화
- langchain_experimental 설치

- https://python.langchain.com/docs/how_to/graph_constructing/#llm-graph-transformer

`(1) LLMGraphTransformer 초기화`

In [ ]:
from typing import List, Dict, Any  
from langchain_openai import ChatOpenAI 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document  

# LLM(대규모 언어 모델) 설정
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# 그래프 데이터베이스에서 사용할 엔티티(노드) 유형 정의
# 뉴스 기사에서 추출할 엔티티 타입을 제한하여 정확도 향상
allowed_nodes = ["Company", "Product", "Technology"]

# 엔티티 간의 관계 유형 정의
# (시작 노드 유형, 관계 유형, 끝 노드 유형) 형태로 허용되는 관계 명시
allowed_relationships = [
    ("Company", "RELEASED", "Product"),     # 회사가 제품을 출시함
    ("Company", "DEVELOPED", "Technology"), # 회사가 기술을 개발함
    ("Product", "USES", "Technology")       # 제품이 기술을 사용함
]

# LLM Graph Transformer 초기화 및 설정
# 텍스트에서 그래프 구조(노드와 관계)를 추출하는 변환기 설정
transformer = LLMGraphTransformer(
    llm=llm,                               # 사용할 언어 모델
    allowed_nodes=allowed_nodes,           # 허용되는 노드 유형
    allowed_relationships=allowed_relationships,  # 허용되는 관계 유형
    node_properties=["industry", "version", "releaseDate", "category"]  # 노드에 추가할 수 있는 속성들
)


`(2) 뉴스 기사에서 그래프 데이터 추출`

In [ ]:
# 뉴스 기사 데이터를 LangChain의 Document 객체로 변환하는 함수
# - LangChain의 Document 객체는 텍스트 콘텐츠와 메타데이터를 함께 저장할 수 있는 구조
# - 이를 통해 LLM이 문서 처리 시 메타데이터 정보도 함께 활용 가능
def convert_to_documents(articles: List[Dict[str, Any]]) -> List[Document]:
    documents = []
    for article in articles:
        # 각 기사마다 Document 객체 생성
        # - page_content: 기사 본문 내용 저장
        # - metadata: 기사의 부가 정보(ID, 제목, 출처, 작성자, 날짜) 저장
        doc = Document(
            page_content=article["content"],
            metadata={
                "id": article["id"],
                "title": article["title"],
                "source": article["source"],
                "author": article["author"],
                "date": article["date"]
            }
        )
        documents.append(doc)
    return documents

# 앞서 추출한 뉴스 기사(extracted_articles)를 LangChain Document 객체 리스트로 변환
# - 이 변환을 통해 LLMGraphTransformer가 처리할 수 있는 형태로 데이터 준비
documents = convert_to_documents(extracted_articles)
print(f"변환된 문서 수: {len(documents)}")
print("=" * 100)

# 변환된 첫 번째 문서의 내용 확인 (디버깅 및 검증 목적)
print(documents[0].page_content[:100])  # 첫 100자만 출력하여 내용 미리보기
print("-" * 100)
print(documents[0].metadata)  # 문서의 메타데이터 전체 출력 (ID, 제목, 출처 등)

변환된 문서 수: 5
구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활
----------------------------------------------------------------------------------------------------
{'id': 'article_0', 'title': '구글, 안드로이드 14에 생성형 AI 기능 대거 탑재', 'source': '디지털타임스', 'author': '박승리', 'date': '2024-03-20'}


In [ ]:
# 뉴스 기사에서 그래프 데이터 추출
# - LLMGraphTransformer를 사용하여 일반 문서를 그래프 구조로 변환
# - 각 문서(뉴스 기사)에서 노드(회사, 기술, 제품 등)와 관계(개발, 출시, 사용 등)를 추출
# - LLM이 텍스트를 분석하여 허용된 노드 유형과 관계 유형에 맞게 구조화된 데이터 생성
# - 추출된 그래프 데이터는 Neo4j에 저장하기 위한 중간 형태로 사용됨

graph_documents = transformer.convert_to_graph_documents(documents)

# 변환된 그래프 문서의 수를 출력하여 처리 결과 확인
print(f"그래프 문서 수: {len(graph_documents)}")

그래프 문서 수: 5


In [ ]:
graph_documents[0].nodes  # 첫 번째 그래프 문서의 노드 정보 확인

[Node(id='구글', type='Company', properties={'industry': 'technology'}),
 Node(id='안드로이드 14', type='Product', properties={'version': '14', 'category': 'mobile operating system'}),
 Node(id='생성형 Ai', type='Technology', properties={'category': 'artificial intelligence'}),
 Node(id='사진 보정 기능', type='Technology', properties={'category': 'image processing'}),
 Node(id='실시간 번역 기능', type='Technology', properties={'category': 'translation'}),
 Node(id='실시간 음성 번역 기능', type='Technology', properties={'category': 'speech translation'}),
 Node(id='구글 어시스턴트', type='Product', properties={'category': 'virtual assistant'})]

In [ ]:
graph_documents[0].relationships  # 첫 번째 그래프 문서의 관계 정보 확인

[Relationship(source=Node(id='구글', type='Company', properties={}), target=Node(id='안드로이드 14', type='Product', properties={}), type='RELEASED', properties={}),
 Relationship(source=Node(id='구글', type='Company', properties={}), target=Node(id='생성형 Ai', type='Technology', properties={}), type='DEVELOPED', properties={}),
 Relationship(source=Node(id='안드로이드 14', type='Product', properties={}), target=Node(id='생성형 Ai', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='안드로이드 14', type='Product', properties={}), target=Node(id='사진 보정 기능', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='안드로이드 14', type='Product', properties={}), target=Node(id='실시간 번역 기능', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='안드로이드 14', type='Product', properties={}), target=Node(id='실시간 음성 번역 기능', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=

`(3) 추출된 그래프 데이터를 Neo4j에 저장`

In [ ]:
# 추출된 그래프 데이터를 Neo4j 데이터베이스에 저장하는 과정
# - 각 그래프 문서(doc)는 LLM이 뉴스 기사에서 추출한 구조화된 정보를 포함

for doc in graph_documents:
    # 디버깅 및 검증을 위해 각 문서에서 추출된 그래프 요소 출력
    # - 노드: 회사, 기술, 제품 등의 개체 정보 (이름, 유형, 속성 등)
    print(f"추출된 노드: {doc.nodes}")
    
    # - 관계: 노드 간의 연결 정보 (개발, 출시, 사용 등의 관계 유형)
    print(f"추출된 관계: {doc.relationships}")
    
    # Neo4j 그래프 데이터베이스에 추출된 정보 저장
    # - add_graph_documents: 그래프 문서를 Neo4j에 저장하는 메서드
    # - [doc]: 단일 문서를 리스트 형태로 전달 (일괄 처리 가능)
    # - include_source=True: 원본 문서 정보도 함께 저장 (추적성 유지)
    graph.add_graph_documents([doc], include_source=True)
    
    print("-" * 100)

추출된 노드: [Node(id='구글', type='Company', properties={'industry': 'technology'}), Node(id='안드로이드 14', type='Product', properties={'version': '14', 'category': 'mobile operating system'}), Node(id='생성형 Ai', type='Technology', properties={'category': 'artificial intelligence'}), Node(id='사진 보정 기능', type='Technology', properties={'category': 'image processing'}), Node(id='실시간 번역 기능', type='Technology', properties={'category': 'translation'}), Node(id='실시간 음성 번역 기능', type='Technology', properties={'category': 'speech translation'}), Node(id='구글 어시스턴트', type='Product', properties={'category': 'virtual assistant'})]
추출된 관계: [Relationship(source=Node(id='구글', type='Company', properties={}), target=Node(id='안드로이드 14', type='Product', properties={}), type='RELEASED', properties={}), Relationship(source=Node(id='구글', type='Company', properties={}), target=Node(id='생성형 Ai', type='Technology', properties={}), type='DEVELOPED', properties={}), Relationship(source=Node(id='안드로이드 14', type='Product', pr

In [ ]:
# Neo4j 데이터베이스에서 Document 노드 레이블을 NewsArticle로 변경하는 작업
# - 그래프 데이터 모델의 일관성을 위해 Document 레이블을 NewsArticle로 변경
# - 뉴스 기사 데이터의 특성을 더 명확하게 표현하기 위한 레이블 변경
# - 이후 분석 및 쿼리 작업에서 뉴스 기사 노드를 더 직관적으로 참조 가능

# Cypher 쿼리 정의:
# - MATCH (d:Document): Document 레이블을 가진 모든 노드 선택
# - SET d:NewsArticle: 선택된 노드에 NewsArticle 레이블 추가
# - REMOVE d:Document: 기존 Document 레이블 제거
# - RETURN count(d) AS count: 변경된 노드 수 반환
cypher_query = """
MATCH (d:Document)
SET d:NewsArticle
REMOVE d:Document
RETURN count(d) AS count
"""

# 정의된 Cypher 쿼리를 Neo4j 데이터베이스에 실행
result = graph.query(cypher_query)

# 변경 작업 결과 출력
# - result[0]['count']: 쿼리 결과에서 변경된 노드 수 추출
print(f"모든 Document 노드 이름을 NewsArticle로 변경했습니다. 변경된 노드 수: {result[0]['count']}")

모든 Document 노드 이름을 NewsArticle로 변경했습니다. 변경된 노드 수: 5


`(4) NewsArticle 관련 노드/관계 생성`

In [ ]:
# 언론사를 Company 노드로 추가하는 Cypher 쿼리
# - MATCH (n:NewsArticle): 모든 NewsArticle 노드를 찾음
# - WITH DISTINCT n.source AS source: 중복 없이 뉴스 기사의 출처(언론사) 추출
# - MERGE (c:Company {name: source, type: 'news'}): 
#   * 해당 이름의 Company 노드가 없으면 생성, 있으면 기존 노드 사용
#   * name 속성에는 언론사 이름, type 속성에는 'news' 값 설정
# - RETURN count(c) AS count: 생성 또는 매칭된 Company 노드 수 반환
query = """
MATCH (n:NewsArticle)
WITH DISTINCT n.source AS source
MERGE (c:Company {name: source, type: 'news'})
RETURN count(c) AS count
"""
# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(query)

# 생성된 언론사 노드 수 출력
print(f"언론사 노드 수: {result[0]['count']}")

언론사 노드 수: 2


In [ ]:
# 뉴스 기사와 언론사 노드 연결하는 Cypher 쿼리
# - MATCH (n:NewsArticle), (c:Company): NewsArticle 노드와 Company 노드를 각각 찾음
# - WHERE n.source = c.name: 뉴스 기사의 source 속성이 회사의 name 속성과 일치하는 경우 필터링
# - MERGE (n)-[:PUBLISHED_BY]->(c): 뉴스 기사에서 언론사로 향하는 PUBLISHED_BY 관계 생성
#   * 관계가 이미 존재하면 기존 관계 사용, 없으면 새로 생성
# - RETURN count(n) AS count: 관계가 생성된 뉴스 기사 노드의 수를 반환
query = """
MATCH (n:NewsArticle), (c:Company)
WHERE n.source = c.name
MERGE (n)-[:PUBLISHED_BY]->(c)
RETURN count(n) AS count
"""
# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(query)
# 생성된 관계 수 출력
print(f"뉴스 기사와 언론사 노드 연결 수: {result[0]['count']}")

뉴스 기사와 언론사 노드 연결 수: 5


In [ ]:
# 기자를 Person 노드로 추가하고, 뉴스 기사 노드와 언론사 노드와 연결하는 Cypher 쿼리
query = """
// 모든 뉴스 기사 노드를 찾음
MATCH (n:NewsArticle)

// 중복 없이 기자(author), 언론사(source), 기사ID를 추출
WITH DISTINCT n.author AS author, n.source AS source, n.id AS articleId

// 기자 노드 생성 (없으면 생성, 있으면 기존 노드 사용)
// MERGE: 노드가 존재하지 않으면 생성하고, 존재하면 매칭함
MERGE (p:Person {name: author})

// 이전 단계에서 생성한 기자 노드(p)와 언론사 정보, 기사ID를 다음 단계로 전달
WITH p, source, articleId

// 언론사 노드 생성 또는 매칭
MERGE (c:Company {name: source})

// 기자와 언론사 사이에 WORKS_FOR 관계 생성
MERGE (p)-[:WORKS_FOR]->(c)

// 기자 노드와 기사ID를 다음 단계로 전달
WITH p, articleId

// 해당 ID를 가진 뉴스 기사 노드 찾기
MATCH (a:NewsArticle {id: articleId})

// 기자와 뉴스 기사 사이에 WROTE 관계 생성
MERGE (p)-[:WROTE]->(a)

// 생성된 고유한 기자 노드의 수를 반환
// count(DISTINCT p): 중복 없이 기자 노드 수를 계산
RETURN count(DISTINCT p) AS count
"""

# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(query)

# 생성된 기자 노드 수 출력
print(f"기자 노드 수: {result[0]['count']}")

기자 노드 수: 5


---

## 3. **쿼리를 이용한 뉴스 데이터 분석**

### 3.1 cypher 구문 직접 사용


#### 1) **스키마 정보 확인**

In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **NewsArticle**
  - `id`: STRING Available options: ['article_0', 'article_1', 'article_2', 'article_3', 'article_4']
  - `text`: STRING Available options: ['구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의', '애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다.', "메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다", "삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적용한 신개념 메모리", "엔비디아가 차세대 AI 전용 GPU 'H200'의 출시가 임박했다고 발표했다. 이번 신제품"]
  - `date`: STRING Available options: ['2024-03-20', '2024-03-19', '2024-04-05', '2024-04-03', '2024-04-02']
  - `source`: STRING Available options: ['디지털타임스', '전자일보']
  - `title`: STRING Available options: ['구글, 안드로이드 14에 생성형 AI 기능 대거 탑재', '애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화', "메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개", "삼성전자, 'AI 메모리' 신기술 개발 성공", "엔비디아, 차세대 AI 전용 GPU 'H200' 출시 임박"]
  - `author`: STRING Available options: ['박승리', '최기상', '정주리', '이승지', '김송이']
- **Company**
  - `id`: STRING Available options: ['구글', 'Apple', '메타', '삼성전자', '엔비디아']
  - `industry`: S

#### 2) **기업 및 제품 언론 보도 분석**

- 목적: 기업과 제품의 미디어 노출 정도 측정
- 유용성: PR 및 마케팅 효과 분석

In [ ]:
# 뉴스 기사에서 특정 기업의 멘션 횟수 분석
# 목적: 각 기업이 뉴스 기사에서 언급된 횟수를 계산하여 미디어 노출도 측정
# 방법: NewsArticle과 Company 노드 간의 MENTIONS 관계를 통해 연결 분석
query = """
// 뉴스 기사에서 기업 언급 횟수 분석
// MATCH: NewsArticle 노드와 Company 노드 간의 MENTIONS 관계 찾기
MATCH (n:NewsArticle)-[:MENTIONS]->(c:Company)

// WITH: 각 기업별로 그룹화하여 언급 횟수 계산
WITH c.id AS companyName, COUNT(n) AS mentionCount

// RETURN: 기업명과 언급 횟수 반환
RETURN companyName, mentionCount

// ORDER BY: 언급 횟수 기준 내림차순 정렬
ORDER BY mentionCount DESC  
"""

# Neo4j 데이터베이스에 쿼리 실행하여 결과 가져오기
result = graph.query(query)

# 각 기업명과 해당 기업이 언급된 기사 수를 내림차순으로 표시
for record in result:
    print(f"{record['companyName']}: {record['mentionCount']}회")

구글: 1회
Apple: 1회
메타: 1회
삼성전자: 1회
엔비디아: 1회


In [ ]:
# 뉴스 기사에서 특정 제품의 멘션 횟수 분석
# 목적: 각 제품이 뉴스 기사에서 언급된 횟수를 계산하여 제품의 미디어 노출도 측정
# 방법: NewsArticle과 Product 노드 간의 MENTIONS 관계를 통해 연결 분석
query = """
// 뉴스 기사에서 제품 언급 횟수 분석
// MATCH: NewsArticle 노드와 Product 노드 간의 MENTIONS 관계 찾기
MATCH (n:NewsArticle)-[:MENTIONS]->(p:Product)

// WITH: 각 제품별로 그룹화하여 언급 횟수 계산
WITH p.id AS productName, COUNT(n) AS mentionCount

// RETURN: 제품명과 언급 횟수 반환
RETURN productName, mentionCount

// ORDER BY: 언급 횟수 기준 내림차순 정렬
ORDER BY mentionCount DESC  
"""
# Neo4j 데이터베이스에 쿼리 실행하여 결과 가져오기
result = graph.query(query)

# 각 제품명과 해당 제품이 언급된 기사 수를 내림차순으로 표시
for record in result:
    print(f"{record['productName']}: {record['mentionCount']}회")

안드로이드 14: 1회
구글 어시스턴트: 1회
Ios 18: 1회
Llama 3: 1회
Ai 메모리: 1회
H200: 1회


#### 3) **뉴스 기사에서 언급된 기술 순위**

- 목적: 미디어에서 가장 많이 언급되는 기술 파악
- 유용성: 기술 트렌드 및 미디어 관심도 분석

In [ ]:
# 뉴스 기사에서 특정 기술의 멘션 횟수 분석
# 목적: 각 기술이 뉴스 기사에서 언급된 횟수를 계산하여 기술의 미디어 노출도 측정
# 방법: NewsArticle과 Technology 노드 간의 MENTIONS 관계를 통해 연결 분석
query = """
// 뉴스 기사에서 기술 언급 횟수 분석
// MATCH: NewsArticle 노드와 Technology 노드 간의 MENTIONS 관계 찾기
MATCH (n:NewsArticle)-[:MENTIONS]->(t:Technology)

// RETURN: 기술 ID와 해당 기술을 언급한 고유 뉴스 기사 수 반환
RETURN t.id, COUNT(DISTINCT n) AS news_coverage

// ORDER BY: 뉴스 기사 수 기준 내림차순 정렬
ORDER BY news_coverage DESC

// LIMIT: 상위 5개 결과만 표시
LIMIT 5
"""

# Neo4j 데이터베이스에 쿼리 실행하여 결과 가져오기
result = graph.query(query)

# 각 기술과 해당 기술이 언급된 기사 수를 내림차순으로 표시
for record in result:
    print(f"{record['t.id']}: {record['news_coverage']}회")

사진 보정 기능: 1회
실시간 번역 기능: 1회
실시간 음성 번역 기능: 1회
Apple Intelligence: 1회
생성형 Ai: 1회


#### 4) **기업별 뉴스 작성자 식별**

- 목적: 기업별 뉴스 작성자 파악
- 유용성: 미디어 관계 및 보도 패턴 분석

In [ ]:
# 특정 기업에 대해 글을 쓴 저자 조회
# 목적: 각 기업을 언급한 뉴스 기사의 작성자를 식별하고 기사 수를 계산
# 방법: NewsArticle-MENTIONS->Company와 Person-WROTE->NewsArticle 관계를 함께 분석
query = """
// 기업과 기사 관계 조회
// MATCH: NewsArticle 노드와 Company 노드 간의 MENTIONS 관계 찾기
// MATCH: NewsArticle 노드와 Person 노드 간의 WROTE 관계 찾기
MATCH (n:NewsArticle)-[:MENTIONS]->(c:Company), 
      (n)<-[:WROTE]-(person:Person)

// RETURN: 기업명, 작성자 목록, 기사 수 반환
RETURN c.id AS companyName,    // 기업명
       COLLECT(DISTINCT person.name) AS authors,   // 작성자 목록 (중복 제거)
       COUNT(DISTINCT n) AS article_count          // 기사 수 (중복 제거)

// ORDER BY: 기사 수 기준 내림차순 정렬
ORDER BY article_count DESC
"""

# Neo4j 데이터베이스에 쿼리 실행하여 결과 가져오기
result = graph.query(query)

# 저자 조회 결과 출력
# 각 기업별로 해당 기업을 언급한 기사의 작성자 목록과 총 기사 수를 표시
for record in result:
    authors = record['authors']
    authors_str = ', '.join(authors)  # 작성자 목록을 쉼표로 구분하여 문자열로 변환
    print(f"{record['companyName']}의 저자: {authors_str} (총 기사 수: {record['article_count']})")

구글의 저자: 박승리 (총 기사 수: 1)
Apple의 저자: 최기상 (총 기사 수: 1)
메타의 저자: 정주리 (총 기사 수: 1)
삼성전자의 저자: 이승지 (총 기사 수: 1)
엔비디아의 저자: 김송이 (총 기사 수: 1)


#### 5) **기업과 보유 기술 조회**

- 목적: 각 기업이 개발한 기술 목록을 확인
- 유용성: 기업의 기술 포트폴리오 파악에 도움

In [ ]:
# 각 기업이 개발한 기술을 조회하는 쿼리
# 목적: 기업과 기술 간의 관계를 파악하여 각 기업의 기술 포트폴리오 확인
# 방법: Company-DEVELOPED->Technology 관계를 통해 기업이 개발한 기술 목록 추출
query = """
// 기업과 기술 관계 조회
// MATCH: Company 노드와 Technology 노드 간의 DEVELOPED 관계 찾기
MATCH (c:Company)-[:DEVELOPED]->(t:Technology)

// RETURN: 기업명과 기술 목록 반환
RETURN c.id AS companyName, COLLECT(t.id) AS technologies
"""

# Neo4j 데이터베이스에 쿼리 실행하여 결과 가져오기
result = graph.query(query)

# 기업과 기술 관계 결과 출력
# 각 기업별로 해당 기업이 개발한 모든 기술을 쉼표로 구분하여 표시
for record in result:
    technologies = record['technologies']
    technologies_str = ', '.join(technologies)  # 기술 목록을 쉼표로 구분하여 문자열로 변환
    print(f"{record['companyName']}이 개발한 기술: {technologies_str}")

구글이 개발한 기술: 생성형 Ai
Apple이 개발한 기술: Apple Intelligence
삼성전자이 개발한 기술: 컴퓨팅 인 메모리


### 3.2 그래프 검색을 위해 **Text2cypher** 사용

- LangChain으로 Neo4J 지식 그래프 조회
- https://python.langchain.com/docs/integrations/graphs/neo4j_cypher/


#### 1) **스키마 정보 확인**

- LLM이 Cypher 쿼리를 생성하려면 그래프 데이터베이스의 스키마 정보가 필요

In [ ]:
# 기본 스키마 정보 확인
graph.refresh_schema()
print(graph.schema)

Node properties:
- **NewsArticle**
  - `id`: STRING Available options: ['article_0', 'article_1', 'article_2', 'article_3', 'article_4']
  - `text`: STRING Available options: ['구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의', '애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다.', "메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다", "삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적용한 신개념 메모리", "엔비디아가 차세대 AI 전용 GPU 'H200'의 출시가 임박했다고 발표했다. 이번 신제품"]
  - `date`: STRING Available options: ['2024-03-20', '2024-03-19', '2024-04-05', '2024-04-03', '2024-04-02']
  - `source`: STRING Available options: ['디지털타임스', '전자일보']
  - `title`: STRING Available options: ['구글, 안드로이드 14에 생성형 AI 기능 대거 탑재', '애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화', "메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개", "삼성전자, 'AI 메모리' 신기술 개발 성공", "엔비디아, 차세대 AI 전용 GPU 'H200' 출시 임박"]
  - `author`: STRING Available options: ['박승리', '최기상', '정주리', '이승지', '김송이']
- **Company**
  - `id`: STRING Available options: ['구글', 'Apple', '메타', '삼성전자', '엔비디아']
  - `industry`: S

#### 2) **GraphCypherQAChain 설정**

- `GraphCypherQAChain`은 LangChain에서 제공하는 체인으로, 자연어 질문을 Cypher 쿼리로 변환하고 그 결과를 바탕으로 답변을 생성

- 작동 과정:
    1. 사용자의 자연어 질문 입력
    2. LLM을 사용하여 질문을 Cypher 쿼리로 변환
    3. 생성된 Cypher 쿼리를 Neo4j 데이터베이스에 실행
    4. 쿼리 결과를 LLM에 전달하여 자연어 답변 생성

- 주요 구성 요소
    - `cypher_generation_chain`: 자연어를 Cypher 쿼리로 변환하는 체인
    - `qa_chain`: 쿼리 결과를 바탕으로 답변을 생성하는 체인
    - `graph`: Neo4j 그래프 데이터베이스 연결 객체
    - `graph_schema`: 그래프 데이터베이스의 스키마 정보

In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_google_genai import ChatGoogleGenerativeAI 

# LLM 모델 설정
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# GraphCypherQAChain 생성
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,  
    validate_cypher=True,            # Cypher 쿼리 유효성 검사
    return_intermediate_steps=True,  # 중간 단계 결과 반환
    allow_dangerous_requests=True,   # DB에 영향을 줄 수 있음을 인지하고 쿼리 실행을 허용
    top_k=5                          # 반환할 최대 결과 수
)

#### 3) **Text to Cypher - DB 조회**

In [ ]:
cypher_query = """
// Company 노드 중 id 속성이 있는 노드 찾기
MATCH (c:Company)
WHERE c.id IS NOT NULL AND c.name IS NULL

// id 값을 name 속성으로 복사하고 id 속성 삭제
SET c.name = c.id
REMOVE c.id

RETURN count(c) AS updated_companies
"""

# Cypher 쿼리 실행
result = graph.query(cypher_query)

In [ ]:
result

[{'updated_companies': 5}]

In [ ]:
graph.refresh_schema()  # 스키마 새로 고침
print(graph.schema)  # 스키마 정보 출력

Node properties:
- **NewsArticle**
  - `id`: STRING Available options: ['article_0', 'article_1', 'article_2', 'article_3', 'article_4']
  - `text`: STRING Available options: ['구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의', '애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다.', "메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다", "삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적용한 신개념 메모리", "엔비디아가 차세대 AI 전용 GPU 'H200'의 출시가 임박했다고 발표했다. 이번 신제품"]
  - `date`: STRING Available options: ['2024-03-20', '2024-03-19', '2024-04-05', '2024-04-03', '2024-04-02']
  - `source`: STRING Available options: ['디지털타임스', '전자일보']
  - `title`: STRING Available options: ['구글, 안드로이드 14에 생성형 AI 기능 대거 탑재', '애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화', "메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개", "삼성전자, 'AI 메모리' 신기술 개발 성공", "엔비디아, 차세대 AI 전용 GPU 'H200' 출시 임박"]
  - `author`: STRING Available options: ['박승리', '최기상', '정주리', '이승지', '김송이']
- **Company**
  - `name`: STRING Available options: ['구글', 'Apple', '메타', '삼성전자', '엔비디아', '디지털타임스', '전자일보

In [ ]:
cypher_chain.invoke({"query": "엔비디아를 언급한 기사는 몇 개인가요?"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on t

KeyboardInterrupt: 

In [ ]:
cypher_chain.invoke({"query": "가장 많은 기사를 작성한 언론사는 어디인가요?"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .


KeyboardInterrupt: 

#### 4) **직접 Cypher 결과 얻기 (LLM 답변 없이)** 

- `return_direct`=True

In [ ]:
# 직접 Cypher 결과 얻기 (LLM 답변 없이)
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True,
    return_direct=True  # LLM 답변 생성 단계 건너뛰기
)

# 각 언론사별로 작성한 기사의 개수를 추출
cypher_chain.invoke({"query": "각 언론사별로 몇 개의 뉴스를 작성했나요?"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .




> Entering new GraphCypherQAChain chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. .
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on 

### 3.3 Graph RAG 구현

- 뉴스 본문을 위한 **벡터 인덱스**를 기존 노트북에 추가
- **벡터 임베딩** 생성 및 Neo4j 저장 기능 구현함
- **벡터 유사도 검색**과 **지식 그래프 결합** 하이브리드 검색 구현
- 기본 **RAG 시스템**과 지식 그래프로 **강화된 RAG 시스템** 구현

#### 1) **벡터 임베딩 모델** 설정

- 뉴스 본문의 벡터화 및 저장을 위한 기초 작업
- 임베딩 모델 설정은 벡터 검색 성능에 직접적 영향을 미침

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#### 2) **벡터 인덱스** 생성

- **news_content_embeddings**라는 이름의 벡터 인덱스를 NewsArticle 노드의 **content_embedding** 필드에 적용함 (필드를 새로 추가)
- 벡터 차원을 **1536차원**으로 설정하여 OpenAI의 text-embedding-3-small 모델과 호환되도록 함

In [ ]:
# 벡터 인덱스 생성
create_vector_index_query = """
CREATE VECTOR INDEX news_content_embeddings IF NOT EXISTS
FOR (n:NewsArticle)
ON n.content_embedding
OPTIONS {indexConfig: {
  `vector.dimensions`: 1536,
  `vector.similarity_function`: 'cosine'
}}
"""

# 벡터 인덱스 생성 쿼리 실행
graph.query(create_vector_index_query)

NameError: name 'graph' is not defined

In [ ]:
# 벡터 인덱스 확인
check_vector_index_query = """
SHOW VECTOR INDEXES
"""
vector_indexes = graph.query(check_vector_index_query)
for index in vector_indexes:
    # 벡터 인덱스 정보 출력
    print(f"Index Name: {index['name']}")
    print(f"Type: {index['type']}")    
    print(f"Property Key: {index['properties']}")
    print("-" * 40)

#### 3) **임베딩 생성 및 저장**

- 뉴스 텍스트에 대해 **OpenAI 임베딩**을 생성하는 과정 수행
- 빈 문자열인 경우 처리를 **건너뛰는** 예외 처리 포함
- 생성된 임베딩을 `db.create.setNodeVectorProperty` 프로시저를 통해 **content_embedding** 속성으로 저장

In [ ]:
# 뉴스 기사 데이터 가져오기
news_query = """
// 뉴스 기사 데이터 가져오기
// MATCH: NewsArticle 노드 조회
// WHERE: 뉴스 기사 내용이 NULL이 아닌 경우 필터링
// RETURN: 기사 ID, 제목, 내용 반환
MATCH (n:NewsArticle)
WHERE n.text IS NOT NULL
RETURN n.id AS id, n.title AS title, n.text AS text
"""
news_articles = graph.query(news_query)

# 배치 크기 설정
BATCH_SIZE = 2

# 임베딩 생성 및 저장 (배치 처리)
for i in range(0, len(news_articles), BATCH_SIZE):
    batch = news_articles[i:i+BATCH_SIZE]
    batch_texts = []
    batch_ids = []
    
    # 배치 데이터 준비
    for article in batch:
        content_text = f"{article['title']}\n\n{article['text']}"
        if content_text.strip(): # 빈 문자열 확인
            batch_texts.append(content_text)
            batch_ids.append(article['id'])
    
    try:
        if batch_texts:
            # 배치 단위로 OpenAI 임베딩 생성
            batch_embeddings = embeddings.embed_documents(batch_texts)
            
            # UNWIND를 사용한 배치 업데이트
            batch_data = [{"id": article_id, "embedding": embedding_vector} 
                         for article_id, embedding_vector in zip(batch_ids, batch_embeddings)]
            
            batch_update_query = """
            // 각 기사의 벡터 임베딩 업데이트
            // UNWIND: 배치 데이터 반복 처리
            // MATCH: NewsArticle 노드 조회
            // CALL: db.create.setNodeVectorProperty 프로시저 호출
            // RETURN: 업데이트된 기사 수 반환
            UNWIND $batch AS item
            MATCH (n:NewsArticle {id: item.id})
            CALL db.create.setNodeVectorProperty(n, 'content_embedding', item.embedding)
            RETURN count(n) as updated
            """
            
            result = graph.query(batch_update_query, params={"batch": batch_data})
            print(f"배치 처리 완료: {i+1}~{min(i+len(batch_texts), len(news_articles))} / {len(news_articles)}, 업데이트됨: {result[0]['updated']}")
    except Exception as e:
        print(f"배치 임베딩 생성 실패 (배치 인덱스 {i}): {str(e)}")

print(f"뉴스 기사 임베딩 업데이트 완료!! 총 {len(news_articles)}개 처리")

#### 4) **Neo4j Graph DB 초기화**

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector
import os

# 임베딩 모델 설정 (인덱싱했던 모델과 동일)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

# Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
vector_store = Neo4jVector.from_existing_index(
    embeddings,  # 사용할 임베딩 모델 지정
    url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
    username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
    password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
    database=os.getenv("NEO4J_DATABASE"),  # Neo4j 데이터베이스 이름
    index_name="news_content_embeddings",  # 뉴스 기사용 벡터 인덱스 이름
    node_label="NewsArticle",  # 뉴스 기사 노드 레이블
    text_node_property="text",  # 텍스트 검색 시 반환할 노드의 속성 (뉴스 내용)
    embedding_node_property="content_embedding"  # 임베딩이 저장된 속성 이름
)

#### 5) **기본 RAG 구현**

In [ ]:
# RAG 시스템 구현
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

# RAG 프롬프트 템플릿 정의
template = """
당신은 최신 기술 뉴스에 대한 지식을 갖춘 전문가 AI 비서입니다.
제공된 뉴스 기사 내용을 바탕으로 질문에 정확하게 답변해 주세요.
뉴스 기사에서 찾을 수 없는 정보에 대해서는 솔직하게 모른다고 답변하세요.

참고할 뉴스 기사:
{context}

질문: {question}

답변:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG 체인 구성
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

rag_chain = {
    "question": RunnablePassthrough(), 
    "context": retriever
    } | prompt | llm | StrOutputParser()

# RAG 실행 
answer = rag_chain.invoke("엔비디아를 언급한 기사에서는 어떤 기술과 제품이 언급되었나요?")
print(answer)

#### 6) **지식 그래프 강화 RAG 구현**

In [ ]:
# 지식 그래프로 강화된 RAG 시스템 
def kg_enhanced_rag(question):
    """지식 그래프 정보로 강화된 RAG 시스템"""
    try:
        # 1. 벡터 검색으로 관련 문서 찾기 
        doc_ids = []
        try:
            docs = vector_store.similarity_search(question, k=2)
            
            # metadata에 'id' 키가 있는지 확인하고 안전하게 추출
            for doc in docs:
                if "id" in doc.metadata:
                    doc_ids.append(doc.metadata["id"])
                elif "title" in doc.metadata:  # id가 없으면 제목으로 대체
                    doc_ids.append(doc.metadata["title"])
            
            # 원본 문서 내용 가져오기
            doc_context = "\n".join([doc.page_content for doc in docs])
            
        except Exception as vector_error:
            print(f"벡터 검색 오류 발생")
         
        # 검색 결과가 없거나 ID를 추출할 수 없는 경우 처리
        if not doc_ids:
            return {
                "query": question,
                "result": "관련 문서를 찾을 수 없습니다. 다른 질문을 시도해보세요.",
                "intermediate_steps": []
            }
        
        # 2. 그래프 검색: 관련 문서에서 언급된 엔티티 및 관계 찾기 
        cypher_query = """
        // 검색 결과와, 일치하는 뉴스 기사 찾기
        MATCH (article:NewsArticle)
        WHERE article.id IN $doc_ids OR article.title IN $doc_ids
        
        // 관련 엔티티 찾기 
        WITH article
        OPTIONAL MATCH (article)-[r1:MENTIONS]->(entity1)
        
        // 결과 수집 및 가공 (직접 관계 위주)
        WITH article, 
            // 직접 관계 위주로 수집
             COLLECT(DISTINCT {
                // 엔티티 타입 추출 (기술, 제품, 회사, 인물 등)
                 type: CASE WHEN entity1 IS NOT NULL THEN LABELS(entity1)[0] ELSE NULL END, 
                 // 엔티티 ID 추출 (기술, 제품, 회사, 인물 등의 고유 식별자)
                 // COALESCE: NULL이 아닌 첫 번째 값을 반환 -> 엔티티 노드의 id 속성이 NULL인 경우 대신 name 속성을 사용
                 id: CASE WHEN entity1 IS NOT NULL THEN COALESCE(entity1.id, entity1.name) ELSE NULL END, 
                 // 관계 타입 추출 (MENTIONS, 예를 들어, 기술이 제품을 사용한다는 관계)
                 rel: TYPE(r1)
             }) AS directRelations
        
        // 최종 결과 반환
        RETURN article.id AS article_id, 
               article.title AS title,
               article.text AS text,
               directRelations
        """
        
        # 그래프 검색 실행 및 결과 처리
        graph_results = graph.query(cypher_query, {"doc_ids": doc_ids})
        print(f"그래프 검색 결과: {graph_results}")
        print("--------------------------------")
        
        # 3. 그래프 정보를 텍스트로 변환
        kg_context = ""
        for record in graph_results:
            kg_context += f"기사: {record['title']}\n"
            
            # 관계 정보 추가
            kg_context += "관련 엔티티:\n"
            for rel in record['directRelations']:
                if rel['id'] is not None and rel['type'] is not None:
                    kg_context += f"- {rel['type']}: {rel['id']} (관계: {rel['rel']})\n"
            
            kg_context += "\n"
        
        # 4. 추가적인 엔티티 관계 탐색 
            try:
                # 직접 관련된 엔티티 ID 추출
                entity_ids = []
                for rel in record['directRelations']:
                    if rel['id'] is not None:
                        entity_ids.append(rel['id'])
                
                if entity_ids:
                    # 엔티티 간 관계 탐색 쿼리
                    entity_query = """
                    MATCH (e1)-[r]->(e2)
                    WHERE e1.id IN $entity_ids OR e1.name IN $entity_ids  // 엔티티 ID 또는 이름이 일치하는 경우
                    RETURN DISTINCT
                           COALESCE(e1.id, e1.name) AS from_entity,  // 엔티티 ID 또는 이름
                           LABELS(e1)[0] AS from_type,  // 엔티티 타입
                           TYPE(r) AS relation,  // 관계 타입
                           COALESCE(e2.id, e2.name) AS to_entity,  // 엔티티 ID 또는 이름
                           LABELS(e2)[0] AS to_type  // 엔티티 타입
                    LIMIT 10
                    """
                    
                    entity_results = graph.query(entity_query, {"entity_ids": entity_ids})
                    print(f"엔티티 관계 검색 결과: {entity_results}")
                    print("--------------------------------")
                    
                    # 엔티티 관계 정보 추가
                    if entity_results:
                        kg_context += "엔티티 간 관계:\n"
                        for relation in entity_results:
                            kg_context += f"- {relation['from_type']} '{relation['from_entity']}' {relation['relation']} {relation['to_type']} '{relation['to_entity']}'\n"
                        
                        kg_context += "\n"
            except Exception as entity_error:
                print(f"엔티티 관계 탐색 오류: {str(entity_error)}")
        
        # 5. 통합 컨텍스트 생성
        combined_context = f"문서 정보:\n{doc_context}\n\n지식 그래프 정보:\n{kg_context}"
        
        # 6. 프롬프트 템플릿 정의 (간소화된 버전)
        kg_template = """
        당신은 최신 기술 뉴스에 대한 지식을 갖춘 전문가 AI 비서입니다.
        제공된 뉴스 기사 내용과 지식 그래프 정보를 바탕으로 질문에 정확하게 답변해 주세요.
        
        지식 그래프는 뉴스 기사에 언급된 기술, 제품, 회사, 인물 간의 관계를 보여줍니다.
        이 관계 정보를 활용하여 더 풍부하고 정확한 답변을 제공하세요.
        
        뉴스 기사나 지식 그래프에서 찾을 수 없는 정보에 대해서는 솔직하게 모른다고 답변하세요.
        답변은 간결하고 명확하게 작성하되, 중요한 세부 정보는 빠짐없이 포함해 주세요.
        
        참고할 정보:
        {context}
        
        질문: {question}
        
        답변:
        """
        
        kg_prompt = PromptTemplate(
            template=kg_template,
            input_variables=["context", "question"]
        )
        
        # 7. RAG 체인 구성 및 실행
        rag_chain = kg_prompt | llm | StrOutputParser()
        result = rag_chain.invoke({
            "question": question, 
            "context": combined_context
        })
        
        # 8. 중간 단계 정보 포함하여 결과 반환
        intermediate_steps = [
            {"context": [dict(record) for record in graph_results]}
        ]
        
        return {
            "query": question,
            "result": result,
            "intermediate_steps": intermediate_steps
        }
    
    except Exception as e:
        # 오류 처리 및 디버깅 정보 반환
        return {
            "query": question,
            "result": f"검색 중 오류가 발생했습니다: {str(e)}",
            "error": str(e)
        }


# 실행 테스트
result = kg_enhanced_rag("AI 기술 동향을 분석해주세요.")
print(result)

In [ ]:
from pprint import pprint

# 중간 단계 출력
for step in result['intermediate_steps']:
    pprint(step)
    print("--------------------------------")

In [ ]:
print(result['result'])